<a href="https://colab.research.google.com/github/abishek982001/Emoji-classifier/blob/master/Happy_vs_Sad_emoji_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"


--2020-05-27 12:16:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.140.128, 2a00:1450:400c:c0c::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.140.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2020-05-27 12:16:22 (84.9 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
import os
import zipfile
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip",'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [0]:
train_happy_dir = os.path.join('/tmp/h-or-s/happy')
train_sad_dir = os.path.join('/tmp/h-or-s/sad')

In [5]:
train_happy_name = os.listdir(train_happy_dir)
print(train_happy_name[:10])
train_sad_name = os.listdir(train_sad_dir)
print(train_sad_name[:10])

['happy2-18.png', 'happy2-11.png', 'happy2-03.png', 'happy2-10.png', 'happy1-04.png', 'happy1-12.png', 'happy1-05.png', 'happy2-04.png', 'happy1-16.png', 'happy1-07.png']
['sad1-17.png', 'sad2-02.png', 'sad2-15.png', 'sad1-18.png', 'sad2-11.png', 'sad2-01.png', 'sad2-14.png', 'sad1-09.png', 'sad2-13.png', 'sad2-08.png']


In [6]:
print('Total training happy images: ', len(train_happy_name))
print('Total training sad images: ', len(train_sad_name))

Total training happy images:  40
Total training sad images:  40


In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>=0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training=True

In [0]:
callbacks = myCallback()

In [0]:
model = keras.Sequential([
                          keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(300,300,3)),
                          keras.layers.MaxPooling2D(2,2),
                          keras.layers.Conv2D(64,(3,3),activation='relu'),
                          keras.layers.MaxPooling2D(2,2),
                          keras.layers.Conv2D(64,(3,3),activation='relu'),
                          keras.layers.MaxPooling2D(2,2),
                          keras.layers.Flatten(),
                          keras.layers.Dense(512,activation='relu'),
                          keras.layers.Dense(1,activation='sigmoid')
])

In [0]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(300,300),
    batch_size=10,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/15
8/8 [==============================] - 1s 80ms/step - loss: 9.7207 - accuracy: 0.5250
Epoch 2/15
8/8 [==============================] - 1s 78ms/step - loss: 0.7002 - accuracy: 0.5875
Epoch 3/15
8/8 [==============================] - 1s 77ms/step - loss: 0.4044 - accuracy: 0.8750
Epoch 4/15
8/8 [==============================] - 1s 78ms/step - loss: 0.4620 - accuracy: 0.8000
Epoch 5/15
8/8 [==============================] - 1s 78ms/step - loss: 0.1155 - accuracy: 0.9375
Epoch 6/15
8/8 [==============================] - 1s 79ms/step - loss: 0.1588 - accuracy: 0.9375
Epoch 7/15
8/8 [==============================] - 1s 77ms/step - loss: 0.2406 - accuracy: 0.9000
Epoch 8/15
8/8 [==============================] - ETA: 0s - loss: 0.0141 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 1s 78ms/step - loss: 0.0141 - accuracy: 1.0000


In [16]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a sad")
  else:
    print(fn + " is a happy")
 

Saving 5_grande.png to 5_grande.png
[1.]
5_grande.png is a sad
